In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

### Extract CSVs into DataFrames

In [2]:
vvr_data_world_file = "/Users/alexanderlorin/code/RAM_project/Resources/vegetarian_restaurants_US_datafiniti_vegetarian_restaurants_US_datafiniti.csv"
vvr_df = pd.read_csv(vvr_data_world_file,index_col=0)
vvr_indexed_df = vvr_df.reset_index(drop=False, inplace=False, col_fill='')
vvr_indexed_df.head(10)

,address,categories,city,claimed,country,cuisines,dateOpened,dateUpdated,descriptions,facebookPageURL,...,menuURL,name,paymentTypes,phones,postalCode,priceRange,province,sic,twitter,websites
0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,NaN,US,NaN,NaN,2014-02-01T04:41:06Z,"Tags: American (New), American, and Trendy Sta...",NaN,...,NaN,Village Inn Restaurant and Bakery,NaN,7277964988,33759,NaN,FL,8423,NaN,NaN
1,605 W Main St,Caf and Sandwich Place,Louisville,NaN,US,"Vegetarian, Delicatessen",NaN,2016-07-23T22:37:24Z,"Atlantic No. 5, Louisville: See 4 unbiased rev...",NaN,...,NaN,Atlantic No. 5,Yes,5028833398,40202,USD 25.00-40.00,KY,NaN,NaN,atlanticno5.com
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,NaN,US,NaN,NaN,2015-11-19T22:22:31Z,"Description: Famous Dave's is a Barbecue , Am...",NaN,...,NaN,Famous Dave's,NaN,2536040340,98373,USD 10.00 - USD 15.00,WA,5610,NaN,http://famousdaves.com
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,NaN,US,"Coffee & Tea, Restaurants, American, Bar, Pu...",NaN,2016-06-20T17:47:10Z,"The Feve, Oberlin: See 170 unbiased reviews of...",NaN,...,/oberlin-oh/mip/the-feve-8940755/menu,The Feve,"mastercard, amex, discover, Yes, visa","[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,NaN,http://www.thefeve.com
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,yellowpages.com,US,"Take Out Restaurants, Asian Restaurants, Resta...",NaN,2016-07-23T23:55:05Z,"[""https://www.zagat.com/r/yalla-mediterranean-...",NaN,...,/pleasant-hill-ca/mip/yalla-mediterranean-5061...,Quiznos,"master card, visa",9258258644,94523,$,CA,NaN,quiznos,http://www.yallamedi.com/
5,2243 Richmond Ave,"English Restaurant, Caf, and Breakfast Spot",Houston,NaN,US,"Coffee Shops, American Restaurants, Restaurant...",NaN,2016-07-23T23:24:12Z,"[""https://www.zagat.com/r/hobbit-cafe-houston""]",NaN,...,/houston-tx/mip/hobbit-cafe-22358589/menu,Hobbit Cafe,"check, discover, cash, amex, all major credit ...","[""7135265460""]","[""77098""]",USD 25.00-40.00,TX,NaN,cafehobbit,http://www.myhobbitcafe.com/
6,8007 State Rt 31,"Pizza, Restaurants",Bridgeport,NaN,US,"New York Style Pizza, Pizza By The Slice, Home...",2011-01-01T12:00:00Z,2015-11-19T22:21:54Z,Check Out Our Full Menu At: www.myfamiliapizza...,NaN,...,NaN,Familia Pizza,NaN,3156339909,13030,NaN,NY,5610,NaN,http://www.myfamiliapizza.com
7,415 Thompson Ln,"Nashville Restaurants, Glencliff Estates, Rest...",Nashville,NaN,US,"African, Ethiopian, Vegetarian",NaN,2013-09-16T03:03:44Z,Tags: nothing yet. tagging it. -->,NaN,...,NaN,Lalibela Ethiopian Cuisine,NaN,6153320710,37211-2212,USD 5 - USD 10,TN,5610,NaN,NaN
8,217 W Beloit St,"Orfordville Restaurants, Italian / Pizza / Veg...",Orfordville,NaN,US,NaN,NaN,2013-09-15T02:05:19Z,"Tags: Italian, Pizza, and Family Style Stats: ...",NaN,...,NaN,Villa Pizza (closed),NaN,6088799929,53576,NaN,WI,5610,NaN,NaN
9,309 N Lamar Blvd,Restaurants,Oxford,NaN,US,"American, Vegetarian, Healthy",NaN,2013-09-18T03:45:30Z,NaN,NaN,...,NaN,The Pita Pit,NaN,6622383535,38655-3201,NaN,MS,5610,NaN,NaN


In [3]:
type(vvr_indexed_df["phones"])

pandas.core.series.Series

In [4]:
vvr_indexed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18155 entries, 0 to 18154
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          17201 non-null  object 
 1   categories       17993 non-null  object 
 2   city             17534 non-null  object 
 3   claimed          820 non-null    object 
 4   country          18155 non-null  object 
 5   cuisines         10468 non-null  object 
 6   dateOpened       264 non-null    object 
 7   dateUpdated      18155 non-null  object 
 8   descriptions     11224 non-null  object 
 9   facebookPageURL  1182 non-null   object 
 10  features         6881 non-null   object 
 11  hours            7973 non-null   object 
 12  images           6576 non-null   object 
 13  isClosed         605 non-null    object 
 14  key              18155 non-null  object 
 15  lat              13704 non-null  float64
 16  languages        8 non-null      object 
 17  long        

### Transform premise DataFrame

In [5]:
# Create a filtered dataframe from specific columns
vvr_cols = ["address", "categories", "city", "country","postalCode", "name", "phones", "priceRange", "province", "websites"]
vvr_transformed= vvr_indexed_df[vvr_cols].copy()

# Rename the column headers
vvr_transformed = vvr_transformed.rename(columns={"postalCode":"postalcode",
                                                  "priceRange": "pricerange",
                                                  "index":"id"
                                                  })



vvr_transformed.head(11)

,address,categories,city,country,postalcode,name,phones,pricerange,province,websites
0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,US,33759,Village Inn Restaurant and Bakery,7277964988,NaN,FL,NaN
1,605 W Main St,Caf and Sandwich Place,Louisville,US,40202,Atlantic No. 5,5028833398,USD 25.00-40.00,KY,atlanticno5.com
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,98373,Famous Dave's,2536040340,USD 10.00 - USD 15.00,WA,http://famousdaves.com
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,44074,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",USD 25.00-40.00,OH,http://www.thefeve.com
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,94523,Quiznos,9258258644,$,CA,http://www.yallamedi.com/
5,2243 Richmond Ave,"English Restaurant, Caf, and Breakfast Spot",Houston,US,"[""77098""]",Hobbit Cafe,"[""7135265460""]",USD 25.00-40.00,TX,http://www.myhobbitcafe.com/
6,8007 State Rt 31,"Pizza, Restaurants",Bridgeport,US,13030,Familia Pizza,3156339909,NaN,NY,http://www.myfamiliapizza.com
7,415 Thompson Ln,"Nashville Restaurants, Glencliff Estates, Rest...",Nashville,US,37211-2212,Lalibela Ethiopian Cuisine,6153320710,USD 5 - USD 10,TN,NaN
8,217 W Beloit St,"Orfordville Restaurants, Italian / Pizza / Veg...",Orfordville,US,53576,Villa Pizza (closed),6088799929,NaN,WI,NaN
9,309 N Lamar Blvd,Restaurants,Oxford,US,38655-3201,The Pita Pit,6622383535,NaN,MS,NaN


In [6]:
state_df = vvr_transformed.dropna()

In [7]:
state_df.province.unique()

array(['KY', 'WA', 'OH', 'CA', 'TX', 'MA', 'UT', 'OK', 'WI', 'VA', 'NY',
       'FL', 'NC', 'AZ', 'LA', 'NJ', 'MI', 'WY', 'PA', 'NH', 'NV', 'CT',
       'MN', 'AR', 'MO', 'GA', 'TN', 'MD', 'OR', 'AL', 'IL', 'VT', 'CO',
       'AK', 'NM', 'DC', 'IN', 'SC', 'SD', 'IA', 'NE', 'ME', 'RI', 'ID',
       'KS', 'MS', 'WV', 'ND', 'MT', 'HI', 'DE', 'AU-WA'], dtype=object)

In [8]:
state_df['province'] = state_df['province'].replace(['AU-WA'],'WA')

<ipython-input-8-eb443f848eef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df['province'] = state_df['province'].replace(['AU-WA'],'WA')


In [9]:
state_df.province.unique()

array(['KY', 'WA', 'OH', 'CA', 'TX', 'MA', 'UT', 'OK', 'WI', 'VA', 'NY',
       'FL', 'NC', 'AZ', 'LA', 'NJ', 'MI', 'WY', 'PA', 'NH', 'NV', 'CT',
       'MN', 'AR', 'MO', 'GA', 'TN', 'MD', 'OR', 'AL', 'IL', 'VT', 'CO',
       'AK', 'NM', 'DC', 'IN', 'SC', 'SD', 'IA', 'NE', 'ME', 'RI', 'ID',
       'KS', 'MS', 'WV', 'ND', 'MT', 'HI', 'DE'], dtype=object)

In [10]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [11]:
state_df.head(11)

,address,categories,city,country,postalcode,name,phones,pricerange,province,websites
1,605 W Main St,Caf and Sandwich Place,Louisville,US,40202,Atlantic No. 5,5028833398,USD 25.00-40.00,KY,atlanticno5.com
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,98373,Famous Dave's,2536040340,USD 10.00 - USD 15.00,WA,http://famousdaves.com
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,44074,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",USD 25.00-40.00,OH,http://www.thefeve.com
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,94523,Quiznos,9258258644,$,CA,http://www.yallamedi.com/
5,2243 Richmond Ave,"English Restaurant, Caf, and Breakfast Spot",Houston,US,"[""77098""]",Hobbit Cafe,"[""7135265460""]",USD 25.00-40.00,TX,http://www.myhobbitcafe.com/
12,145 Charles St,"Chinese Restaurants, Asian Restaurants, Take O...",Boston,US,2114,King I,6172273320,$$,MA,http://www.kingandi-boston.com
19,1912 W 5600 S,"[""Tacos"",""Food"",""Mexican"",""Roy Restaurants"",""M...",Roy,US,"[""84067""]",Taco Del Mar,"[""8018258226""]",USD 0.00 - USD 10.00,UT,tacodelmar.com
22,Po Box 6514,caterers,Moore,US,73153,Michael's Catering,4055560377,$$,OK,http://www.michaelcookcatering.com
24,222 E Main St,Deli / Bodega,Port Washington,US,53074,Dockside Deli,"[""2622849440""]",USD 0.00-25.00,WI,docksidedeli.com
25,14810 Detroit Ave,"American Restaurant, Restaurant, Gastropub, Ba...",Lakewood,US,44107,Deagan's Kitchen & Bar,"2167675775, (216) 767-5775",USD 25.00-40.00,OH,deagans.com


In [15]:
phones = state_df['phones'].str.extract(r'(\d{10})')
phones[phones[0]==np.nan]
idx = phones[0].str.len()!=10
phones.isnull().sum()

0    30
dtype: int64

In [20]:
state_df['col11'] = phones

<ipython-input-20-244ad902aa20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df['col11'] = phones


In [21]:
state_df

,address,categories,city,country,postalcode,name,phones,pricerange,province,websites,col11
1,605 W Main St,Caf and Sandwich Place,Louisville,US,40202,Atlantic No. 5,5028833398,USD 25.00-40.00,KY,atlanticno5.com,5028833398
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,98373,Famous Dave's,2536040340,USD 10.00 - USD 15.00,WA,http://famousdaves.com,2536040340
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,44074,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",USD 25.00-40.00,OH,http://www.thefeve.com,4407741978
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,94523,Quiznos,9258258644,$,CA,http://www.yallamedi.com/,9258258644
5,2243 Richmond Ave,"English Restaurant, Caf, and Breakfast Spot",Houston,US,"[""77098""]",Hobbit Cafe,"[""7135265460""]",USD 25.00-40.00,TX,http://www.myhobbitcafe.com/,7135265460
...,...,...,...,...,...,...,...,...,...,...,...
18101,4 E Madison St,"Restaurants, Baltimore Restaurants, Mount Vern...",Baltimore,US,21202,Feast @ 4 East,4106052020,USD 30 - USD 40,MD,http://www.4eastmadisoninn.com,4106052020
18102,7334 Tylers Corner Dr,"Asian, Sushi, Thai, Restaurant, West Chester R...",W Chester,US,45069,Miso - West Chester,5137791807,USD 25.00 - USD 40.00,OH,http://www.misocincinnati.com/,5137791807
18107,3643 S Baldwin Rd,"Take Out Restaurants, Sushi Bars, Bars, Asian ...",Lake Orion,US,"[""48359""]",Big Wave Sushi,"[""12484816991"",""2484816991""]",$,MI,http://www.bigwavesushi.com,1248481699
18148,1839 Winderly Ln,"Greek, Mediterranean, Vegetarian",Pickerington,US,43147,Feta Greek Kuzina,6148643382,USD 10.00 - USD 15.00,OH,http://www.bestgreekusa.com/,6148643382


In [23]:
state_df = state_df.rename(columns={"col11":"new_phones"})
state_df

,address,categories,city,country,postalcode,name,phones,pricerange,province,websites,new_phones
1,605 W Main St,Caf and Sandwich Place,Louisville,US,40202,Atlantic No. 5,5028833398,USD 25.00-40.00,KY,atlanticno5.com,5028833398
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,98373,Famous Dave's,2536040340,USD 10.00 - USD 15.00,WA,http://famousdaves.com,2536040340
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,44074,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",USD 25.00-40.00,OH,http://www.thefeve.com,4407741978
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,94523,Quiznos,9258258644,$,CA,http://www.yallamedi.com/,9258258644
5,2243 Richmond Ave,"English Restaurant, Caf, and Breakfast Spot",Houston,US,"[""77098""]",Hobbit Cafe,"[""7135265460""]",USD 25.00-40.00,TX,http://www.myhobbitcafe.com/,7135265460
...,...,...,...,...,...,...,...,...,...,...,...
18101,4 E Madison St,"Restaurants, Baltimore Restaurants, Mount Vern...",Baltimore,US,21202,Feast @ 4 East,4106052020,USD 30 - USD 40,MD,http://www.4eastmadisoninn.com,4106052020
18102,7334 Tylers Corner Dr,"Asian, Sushi, Thai, Restaurant, West Chester R...",W Chester,US,45069,Miso - West Chester,5137791807,USD 25.00 - USD 40.00,OH,http://www.misocincinnati.com/,5137791807
18107,3643 S Baldwin Rd,"Take Out Restaurants, Sushi Bars, Bars, Asian ...",Lake Orion,US,"[""48359""]",Big Wave Sushi,"[""12484816991"",""2484816991""]",$,MI,http://www.bigwavesushi.com,1248481699
18148,1839 Winderly Ln,"Greek, Mediterranean, Vegetarian",Pickerington,US,43147,Feta Greek Kuzina,6148643382,USD 10.00 - USD 15.00,OH,http://www.bestgreekusa.com/,6148643382


In [25]:
del state_df["phones"]

KeyError: 'phones'

In [26]:
state_df

,address,categories,city,country,postalcode,name,pricerange,province,websites,new_phones
1,605 W Main St,Caf and Sandwich Place,Louisville,US,40202,Atlantic No. 5,USD 25.00-40.00,KY,atlanticno5.com,5028833398
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,98373,Famous Dave's,USD 10.00 - USD 15.00,WA,http://famousdaves.com,2536040340
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,44074,The Feve,USD 25.00-40.00,OH,http://www.thefeve.com,4407741978
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,94523,Quiznos,$,CA,http://www.yallamedi.com/,9258258644
5,2243 Richmond Ave,"English Restaurant, Caf, and Breakfast Spot",Houston,US,"[""77098""]",Hobbit Cafe,USD 25.00-40.00,TX,http://www.myhobbitcafe.com/,7135265460
...,...,...,...,...,...,...,...,...,...,...
18101,4 E Madison St,"Restaurants, Baltimore Restaurants, Mount Vern...",Baltimore,US,21202,Feast @ 4 East,USD 30 - USD 40,MD,http://www.4eastmadisoninn.com,4106052020
18102,7334 Tylers Corner Dr,"Asian, Sushi, Thai, Restaurant, West Chester R...",W Chester,US,45069,Miso - West Chester,USD 25.00 - USD 40.00,OH,http://www.misocincinnati.com/,5137791807
18107,3643 S Baldwin Rd,"Take Out Restaurants, Sushi Bars, Bars, Asian ...",Lake Orion,US,"[""48359""]",Big Wave Sushi,$,MI,http://www.bigwavesushi.com,1248481699
18148,1839 Winderly Ln,"Greek, Mediterranean, Vegetarian",Pickerington,US,43147,Feta Greek Kuzina,USD 10.00 - USD 15.00,OH,http://www.bestgreekusa.com/,6148643382


In [9]:
vvr_transformed["name"].str.contains("close").count()

17333

In [10]:
vvr_transformed.iloc[5,:]["phones"].replace("]","")

'["7135265460"'

In [66]:
state_df

,0,col10
1,5028833398,5028833398
2,2536040340,2536040340
3,4407741978,4407741978
4,9258258644,9258258644
5,7135265460,7135265460
...,...,...
18101,4106052020,4106052020
18102,5137791807,5137791807
18107,1248481699,1248481699
18148,6148643382,6148643382


In [61]:
phones_df

,0
1,5028833398
2,2536040340
3,4407741978
4,9258258644
5,7135265460
...,...
18101,4106052020
18102,5137791807
18107,1248481699
18148,6148643382


In [65]:
state_df = state_df['col10'] = phones_df 
state_df

,0,col10
1,5028833398,5028833398
2,2536040340,2536040340
3,4407741978,4407741978
4,9258258644,9258258644
5,7135265460,7135265460
...,...,...
18101,4106052020,4106052020
18102,5137791807,5137791807
18107,1248481699,1248481699
18148,6148643382,6148643382


In [60]:
phones_df = state_df['phones'].str.extract(r'(\d{10})')
phones_df[phones_df[0]==np.nan]
idx = phones_df[0].str.len()!=10
phones.isnull()

,0
1,False
2,False
3,False
4,False
5,False
...,...
18101,False
18102,False
18107,False
18148,False


In [12]:
vvr_transformed

,address,categories,city,country,postalcode,name,phones,pricerange,province,websites
0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,US,33759,Village Inn Restaurant and Bakery,7277964988,NaN,FL,NaN
1,605 W Main St,Caf and Sandwich Place,Louisville,US,40202,Atlantic No. 5,5028833398,USD 25.00-40.00,KY,atlanticno5.com
2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,98373,Famous Dave's,2536040340,USD 10.00 - USD 15.00,WA,http://famousdaves.com
3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,44074,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",USD 25.00-40.00,OH,http://www.thefeve.com
4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,94523,Quiznos,9258258644,$,CA,http://www.yallamedi.com/
...,...,...,...,...,...,...,...,...,...,...
18150,879 Smith Ave S,"West St Paul Restaurants, American / Diner / C...",W St Paul,US,55118,The Old Man River Cafe,6.51E+09,NaN,MN,NaN
18151,1410 Colonial Life Blvd W,Restaurant,Columbia,US,29210-7665,The Melting Pot,8037318500,NaN,SC,NaN
18152,9439 Sheridan St,"American, Smoothies & Juices, Hollywood Restau...",Hollywood,US,33024,Smoothie King,9544313411,NaN,FL,http://smoothieking.com
18153,801 N Congress Ave,"[""Restaurants"",""Banquet Halls & Reception Faci...",Boynton Beach,US,"[""33426""]",Buca Di Beppo,"[""5617344558"",""5617330189""]",$$,FL,http://www.bucadibeppo.com


In [23]:
state_df.province.unique()

array([], dtype=object)

### Transform county DataFrame

### Create database connection

In [14]:
connection_string = "postgres:postgres@localhost:5432/vegan_ram"
engine = create_engine(f'postgresql://{connection_string}')

In [15]:
# Confirm tables
engine.table_names()

<ipython-input-15-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['vvr']

### Load DataFrames into database

In [16]:
vvr_transformed.to_sql(name='vvr', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "postalcode" of relation "vvr" does not exist
LINE 1: ...O vvr (index, address, categories, city, country, postalcode...
                                                             ^

[SQL: INSERT INTO vvr (index, address, categories, city, country, postalcode, name, phones, pricerange, province, websites) VALUES (%(index)s, %(address)s, %(categories)s, %(city)s, %(country)s, %(postalcode)s, %(name)s, %(phones)s, %(pricerange)s, %(province)s, %(websites)s)]
[parameters: ({'index': 0, 'address': '2871 Gulf To Bay Blvd', 'categories': 'American, American (new), Clearwater Restaurants, American (new) / American / Vegan', 'city': 'Clearwater', 'country': 'US', 'postalcode': '33759', 'name': 'Village Inn Restaurant and Bakery', 'phones': '7277964988', 'pricerange': None, 'province': 'FL', 'websites': None}, {'index': 1, 'address': '605 W Main St', 'categories': 'Caf and Sandwich Place', 'city': 'Louisville', 'country': 'US', 'posta

In [ ]:
# county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)